In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import robot as Robot
import  time

robot_Instance= Robot.Robot_Class(0,0,0)


In [3]:
robot_Instance.Connect_TCP()

Trying to connect
Connected


# Normal Run 

In [4]:
Init=[2000,200,90,15]
robot_Instance.Set_Robot_Position(Init[0],Init[1],Init[2],Transmission=True)
robot_Instance.Start_Collector()
robot_Instance.Set_Max_Try_Single_Goal(12)
robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To_Speed(600,640,20000, Detection=True)#Follow X coordinate
robot_Instance.Go_To_Speed(600,640,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7600,Y_Cal=200,A_Cal=0, Detection=False, Calibration=True)#Recalibrate Top of the ramp
robot_Instance.Go_To(7000,200,1,Path_Checking=False,Avoidance=False)#Backward
robot_Instance.Go_To(7800,1000,0,Path_Checking=False,Avoidance=False)#Turn left

robot_Instance.Go_To_Speed(600,640,20000, Detection=True)#Follow Y coordintae

robot_Instance.Go_To_Speed(600,640,20000, Detection=True)
print(robot_Instance.Get_Robot_Position())

robot_Instance.Go_To_Speed(100,100,800,X_Cal=7500,Y_Cal=7700,A_Cal=90, Detection=False, Calibration=True)#Recalibrate Top of the ramp, Step1
robot_Instance.Disable_All_Sensors()
robot_Instance.Go_To(7500,7200,1,Path_Checking=False,Avoidance=False)#Backward
robot_Instance.Go_To(7000,7200,0,Path_Checking=False,Avoidance=False)#Move backward right so we can recalibrate at the top of the ramp
robot_Instance.Go_To_Speed(-320,-300,10000, Detection=True)#Go against wall
robot_Instance.Go_To_Speed(-100,-100,1500,X_Cal=7900,A_Cal=180, Detection=False, Calibration=True)#Recalibrate Corner paltform
robot_Instance.Go_To(5400,7500,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,7000,0,Path_Checking=False,Avoidance=False)#In front of stairs
robot_Instance.Go_To(7000,6500,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,6800,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,7200,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7800,7800,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(420,400,20000, Detection=True)#Calibrate Top of the ramp
robot_Instance.Go_To_Speed(100,100,1200,X_Cal=7700,Y_Cal=7750,A_Cal=0, Detection=False, Calibration=True)

robot_Instance.Go_To(7300,7750,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7600,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7800,7300,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(120,100,10000, Detection=True)#Downhill
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7800,Y_Cal=300,A_Cal=-90, Detection=False, Calibration=True)
robot_Instance.Go_To(7800,700,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,500,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(100,100,800,X_Cal=200,Y_Cal=300,A_Cal=180, Detection=False, Calibration=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)

robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1400,300,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
robot_Instance.Go_To(1700,300,1,Path_Checking=False,Avoidance=False)
"""End of RAMP"""
robot_Instance.Start_Collector()

robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Enable_All_Sensors()
time.sleep(0.2)
robot_Instance.Clear_Read_Buffer()

robot_Instance.Go_To(1700,750,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1000,1000,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(500,500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)
robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
"""End of Grass"""
while True:
    robot_Instance.Empty_TCP()
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Set_Max_Try_Single_Goal(12)
    robot_Instance.Start_Collector()
    Spot_List =[[1000,2600,0,15],[6500,2600,0,15],[6500,5000,0,15],[4500,5000,0,15],[4500,7500,0,15],[3800,7500,0,15],[3800,5500,0,15],[7500,5500,0,15],[7500,2500,0,15],[3500,2500,0,15],[2500,500,0,15],[2500,4000,0,15],[7000,4000,0,15],[7000,3000,0,15],[3500,3000,0,15],[3500,1500,0,15],[700,3500,0,15],[5500,3500,0,15],[5500,4200,0,15],[700,4200,0,15]]
    for Spot in Spot_List:
        robot_Instance.Update_Robot_Position_From_Beacon()
        Bottle_N=0
        robot_Instance.Sensor_Enabled=Spot[3]
        print('Spot : ', Spot)
        robot_Instance.Clear_Read_Buffer()
        robot_Instance.Clear_Bottle_List()
        robot_Instance.Go_To(Spot[0],Spot[1],Spot[2])
        #Robot is now arrived to a position
        Data=robot_Instance.Get_Data_TCP()#Collect bottles from TCP Communication
        print('Data Received TCP:', Data)
        while Bottle_N<3:#Try to catch 3 Bottle
            List=robot_Instance.Update_Bottle_List(Data)#Update our internal lsit
            Bottle_List= robot_Instance.Get_Bottle_List()
            if Bottle_List!=[]:
                Bottle_X, Bottle_Y, Bottle_Score= robot_Instance.Get_Nearest_Bottle()
                if Bottle_X!=None and Bottle_Y!= None and Bottle_Score!=None:
                    Bottle_X_Extended, Bottle_Y_Extended, Valid=robot_Instance.Extend_Coordinate((Bottle_X, Bottle_Y),Distance=800)
                    print('Bottle selected:', Bottle_X,Bottle_Y,Bottle_Score, ' Extended', Bottle_X_Extended,Bottle_Y_Extended)
                    if Valid:
                        robot_Instance.Go_To(Bottle_X,Bottle_Y,Bottle_Score)
                        robot_Instance.Add_Bottle_Collected()
                        Bottle_N=Bottle_N+1
                    else:
                        robot_Instance.Clean_Bottle_Of_Position((Bottle_X, Bottle_Y))
                else:# No more bottle
                    print('No bottle')
                    break;
            else:
                print("Empty List")
                break;
        if(robot_Instance.Get_Bottle_Collected()>8):
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(1000,1000,0)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(500,500,0, Avoidance=False)
            robot_Instance.Set_Robot_Speed(150, Transmission= True)
            robot_Instance.Reverse_Collector()
            robot_Instance.Clear_Read_Buffer()
            robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
            robot_Instance.Set_Robot_Speed(500, Transmission= True)
            robot_Instance.Start_Collector()
            robot_Instance.Clear_Read_Buffer()
            #Reverse collector
            robot_Instance.Reset_Bottle_Collected()

    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(1000,1000,0)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(500,500,0, Avoidance=False)
    robot_Instance.Set_Robot_Speed(150, Transmission= True)
    robot_Instance.Reverse_Collector()
    robot_Instance.Clear_Read_Buffer()
    robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Clear_Read_Buffer()
    #Reverse collector
    robot_Instance.Reset_Bottle_Collected()

G92 X2000 Y200 A90 

('G1 R600 L640 T20000\r\n', 'M3 H2 S15\r\n')
Message OK: D_Profil S=500 A=50 B=100 

Message  M0 H2 T2

answer Blocked
('G1 R600 L640 T20000\r\n', 'M3 H2 S15\r\n')
Message  M0 H2 T2

answer Blocked
('G1 R100 L100 T800\r\n', 'M3 H2 S0\r\n')
Message  M0 H2 T1

answer Timeout
Setting pos
G92 X7600 Y200 A0 

{'b': 0, 'i': 1, 'h': 0, 'f': 1, 'e': 1, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X7212.81 Y183.61 A2.33 T1 S304

Timeout
Try Again
{'b': 0, 'i': 1, 'h': 0, 'f': 1, 'e': 1, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X7106.81 Y180.63 A0.79 T1 S304

Timeout
{'b': 0, 'i': 1, 'h': 0, 'f': 0, 'e': 1, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X7535.83 Y428.63 A58.27 T1 S272

Timeout
Try Again
{'b': 0, 'i': 1, 'h': 0, 'f': 0, 'e': 1, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X7540.76 Y436.84 A60.35 T2 S272

Blocked
('G1 R600 L640 T20000\r

{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 1, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X6023.78 Y803.54 A-1.02 T2 S2048

Blocked
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X5889.08 Y810.39 A-2.09 T1 S0

Timeout
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X5882.59 Y810.63 A-2.36 T1 S0

Timeout
Try Again
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X5999.70 Y856.63 A45.95 T1 S0

Timeout
Try Again
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X6049.19 Y987.76 A89.05 T1 S0

Timeout
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X6680.65 Y1052.14 A-29.32 T1 S0

Timeout
Try Again
{'b': 0, 'i': 0, 'h': 0, 'f':

{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X5872.32 Y3744.90 A-17.20 T1 S0

Timeout
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X6031.72 Y3808.33 A53.03 T1 S0

Timeout
Data Received TCP: None
Empty List
total time capture image:  0.6646988391876221
r  not found.
[-159.4039716888516, -111.87620811163073, -76.67546873810922] [(8000, 0), (0, 0), (8000, 8000)]
Beacon worked!
total time post math:  1.5820350646972656
G92 X4138 Y1760 A135 

4137.712179859012 1759.6887962004937 134.90958760745696
Spot :  [4500, 5000, 0, 15]
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 1, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X4028.01 Y2291.08 A83.08 T2 S16

Blocked
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X4002.07 Y2137.95 A84.16 T1 S0

Timeout
{'b': 0, 'i': 0

{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 1, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 1}
Message  M0 X3459.61 Y2817.81 A104.31 T2 S4160

Blocked
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X3502.33 Y2654.80 A102.01 T1 S0

Timeout
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 1, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X3551.10 Y2786.61 A37.92 T1 S4096

Timeout
Try Again
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 1, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X3550.25 Y2785.92 A32.78 T2 S4096

Blocked
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X3407.08 Y2697.97 A33.54 T1 S0

Timeout
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X3575.28 Y2679.69 A-36.42 T1 S0

Timeout
Try Again
{'b': 0, 'i': 0, 'h': 0, '

{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 1, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 1, 'g': 0}
Message  M0 X3895.58 Y3002.46 A213.90 T2 S9

Blocked
Avoidance H
{'b': 1, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 1, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 1, 'g': 0}
Message  M0 X3895.58 Y3002.46 A213.90 T2 S11

Blocked
Avoidance H
Data Received TCP: ['3085.0,2484.0', '3464.0,2484.0', '3240.0,2553.0', '3539.0,2502.0', '3378.0,2855.0', '3393.0,2933.0']
Bottle selected: 3378.0 2855.0 646.1616823420135  Extended 2608.616140839889 2635.8003712049344
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X3552.70 Y2877.84 A188.91 T1 S0

Timeout
Bottle selected: 3378.0 2855.0 184.95562997673335  Extended 2584.7505962409123 2751.2918352498127
Bottle selected: 3378.0 2855.0 184.95562997673335  Extended 2584.7505962409123 2751.2918352498127
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0

KeyboardInterrupt: 

# Reset B(Following Y Wall failed, or climbing)

In [ ]:

robot_Instance.Go_To_Speed(600,640,20000, Detection=True)#Follow Y coordintae

robot_Instance.Go_To_Speed(600,640,20000, Detection=True)
print(robot_Instance.Get_Robot_Position())

robot_Instance.Go_To_Speed(100,100,800,X_Cal=7500,Y_Cal=7700,A_Cal=90, Detection=False, Calibration=True)#Recalibrate Top of the ramp, Step1
robot_Instance.Disable_All_Sensors()
robot_Instance.Go_To(7500,7200,1,Path_Checking=False,Avoidance=False)#Backward
robot_Instance.Go_To(7000,7200,0,Path_Checking=False,Avoidance=False)#Move backward right so we can recalibrate at the top of the ramp
robot_Instance.Go_To_Speed(-320,-300,10000, Detection=True)#Go against wall
robot_Instance.Go_To_Speed(-100,-100,1500,X_Cal=7900,A_Cal=180, Detection=False, Calibration=True)#Recalibrate Corner paltform
robot_Instance.Go_To(5400,7500,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,7000,0,Path_Checking=False,Avoidance=False)#In front of stairs
robot_Instance.Go_To(7000,6500,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,6800,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,7200,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7800,7800,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(420,400,20000, Detection=True)#Calibrate Top of the ramp
robot_Instance.Go_To_Speed(100,100,1200,X_Cal=7700,Y_Cal=7750,A_Cal=0, Detection=False, Calibration=True)

robot_Instance.Go_To(7300,7750,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7600,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7800,7300,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(120,100,10000, Detection=True)#Downhill
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7800,Y_Cal=300,A_Cal=-90, Detection=False, Calibration=True)
robot_Instance.Go_To(7800,700,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,500,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(100,100,800,X_Cal=200,Y_Cal=300,A_Cal=180, Detection=False, Calibration=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)

robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1400,300,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
robot_Instance.Go_To(1700,300,1,Path_Checking=False,Avoidance=False)
"""End of RAMP"""
robot_Instance.Start_Collector()

robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Enable_All_Sensors()
time.sleep(0.2)
robot_Instance.Clear_Read_Buffer()

robot_Instance.Go_To(1700,750,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1000,1000,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(500,500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)
robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
"""End of Grass"""
while True:
    robot_Instance.Empty_TCP()
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Set_Max_Try_Single_Goal(12)
    robot_Instance.Start_Collector()
    Spot_List =[[1000,2600,0,15],[6500,2600,0,15],[6500,5000,0,15],[4500,5000,0,15],[4500,7500,0,15],[3800,7500,0,15],[3800,5500,0,15],[7500,5500,0,15],[7500,2500,0,15],[3500,2500,0,15],[2500,500,0,15],[2500,4000,0,15],[7000,4000,0,15],[7000,3000,0,15],[3500,3000,0,15],[3500,1500,0,15],[700,3500,0,15],[5500,3500,0,15],[5500,4200,0,15],[700,4200,0,15]]
    for Spot in Spot_List:
        robot_Instance.Update_Robot_Position_From_Beacon()
        Bottle_N=0
        robot_Instance.Sensor_Enabled=Spot[3]
        print('Spot : ', Spot)
        robot_Instance.Clear_Read_Buffer()
        robot_Instance.Clear_Bottle_List()
        robot_Instance.Go_To(Spot[0],Spot[1],Spot[2])
        #Robot is now arrived to a position
        Data=robot_Instance.Get_Data_TCP()#Collect bottles from TCP Communication
        print('Data Received TCP:', Data)
        while Bottle_N<3:#Try to catch 3 Bottle
            List=robot_Instance.Update_Bottle_List(Data)#Update our internal lsit
            Bottle_List= robot_Instance.Get_Bottle_List()
            if Bottle_List!=[]:
                Bottle_X, Bottle_Y, Bottle_Score= robot_Instance.Get_Nearest_Bottle()
                if Bottle_X!=None and Bottle_Y!= None and Bottle_Score!=None:
                    Bottle_X_Extended, Bottle_Y_Extended, Valid=robot_Instance.Extend_Coordinate((Bottle_X, Bottle_Y),Distance=800)
                    print('Bottle selected:', Bottle_X,Bottle_Y,Bottle_Score, ' Extended', Bottle_X_Extended,Bottle_Y_Extended)
                    if Valid:
                        robot_Instance.Go_To(Bottle_X,Bottle_Y,Bottle_Score)
                        robot_Instance.Add_Bottle_Collected()
                        Bottle_N=Bottle_N+1
                    else:
                        robot_Instance.Clean_Bottle_Of_Position((Bottle_X, Bottle_Y))
                else:# No more bottle
                    print('No bottle')
                    break;
            else:
                print("Empty List")
                break;
        if(robot_Instance.Get_Bottle_Collected()>8):
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(1000,1000,0)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(500,500,0, Avoidance=False)
            robot_Instance.Set_Robot_Speed(150, Transmission= True)
            robot_Instance.Reverse_Collector()
            robot_Instance.Clear_Read_Buffer()
            robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
            robot_Instance.Set_Robot_Speed(500, Transmission= True)
            robot_Instance.Start_Collector()
            robot_Instance.Clear_Read_Buffer()
            #Reverse collector
            robot_Instance.Reset_Bottle_Collected()

    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(1000,1000,0)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(500,500,0, Avoidance=False)
    robot_Instance.Set_Robot_Speed(150, Transmission= True)
    robot_Instance.Reverse_Collector()
    robot_Instance.Clear_Read_Buffer()
    robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Clear_Read_Buffer()
    #Reverse collector
    robot_Instance.Reset_Bottle_Collected()

# Reset C(Platform Movement failed)

In [43]:
robot_Instance.Go_To_Speed(420,400,20000, Detection=True)#Calibrate Top of the ramp
robot_Instance.Go_To_Speed(100,100,1200,X_Cal=7700,Y_Cal=7750,A_Cal=0, Detection=False, Calibration=True)

robot_Instance.Go_To(7300,7750,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7600,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7800,7300,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(120,100,10000, Detection=True)#Downhill
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7800,Y_Cal=300,A_Cal=-90, Detection=False, Calibration=True)
robot_Instance.Go_To(7800,700,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,500,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(100,100,800,X_Cal=200,Y_Cal=300,A_Cal=180, Detection=False, Calibration=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)

robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1400,300,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
robot_Instance.Go_To(1700,300,1,Path_Checking=False,Avoidance=False)
"""End of RAMP"""
robot_Instance.Start_Collector()

robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Enable_All_Sensors()
time.sleep(0.2)
robot_Instance.Clear_Read_Buffer()

robot_Instance.Go_To(1700,750,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1000,1000,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(500,500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)
robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
"""End of Grass"""
while True:
    robot_Instance.Empty_TCP()
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Set_Max_Try_Single_Goal(12)
    robot_Instance.Start_Collector()
    Spot_List =[[1000,2600,0,15],[6500,2600,0,15],[6500,5000,0,15],[4500,5000,0,15],[4500,7500,0,15],[3800,7500,0,15],[3800,5500,0,15],[7500,5500,0,15],[7500,2500,0,15],[3500,2500,0,15],[2500,500,0,15],[2500,4000,0,15],[7000,4000,0,15],[7000,3000,0,15],[3500,3000,0,15],[3500,1500,0,15],[700,3500,0,15],[5500,3500,0,15],[5500,4200,0,15],[700,4200,0,15]]
    for Spot in Spot_List:
        robot_Instance.Update_Robot_Position_From_Beacon()
        Bottle_N=0
        robot_Instance.Sensor_Enabled=Spot[3]
        print('Spot : ', Spot)
        robot_Instance.Clear_Read_Buffer()
        robot_Instance.Clear_Bottle_List()
        robot_Instance.Go_To(Spot[0],Spot[1],Spot[2])
        #Robot is now arrived to a position
        Data=robot_Instance.Get_Data_TCP()#Collect bottles from TCP Communication
        print('Data Received TCP:', Data)
        while Bottle_N<3:#Try to catch 3 Bottle
            List=robot_Instance.Update_Bottle_List(Data)#Update our internal lsit
            Bottle_List= robot_Instance.Get_Bottle_List()
            if Bottle_List!=[]:
                Bottle_X, Bottle_Y, Bottle_Score= robot_Instance.Get_Nearest_Bottle()
                if Bottle_X!=None and Bottle_Y!= None and Bottle_Score!=None:
                    Bottle_X_Extended, Bottle_Y_Extended, Valid=robot_Instance.Extend_Coordinate((Bottle_X, Bottle_Y),Distance=800)
                    print('Bottle selected:', Bottle_X,Bottle_Y,Bottle_Score, ' Extended', Bottle_X_Extended,Bottle_Y_Extended)
                    if Valid:
                        robot_Instance.Go_To(Bottle_X,Bottle_Y,Bottle_Score)
                        robot_Instance.Add_Bottle_Collected()
                        Bottle_N=Bottle_N+1
                    else:
                        robot_Instance.Clean_Bottle_Of_Position((Bottle_X, Bottle_Y))
                else:# No more bottle
                    print('No bottle')
                    break;
            else:
                print("Empty List")
                break;
        if(robot_Instance.Get_Bottle_Collected()>8):
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(1000,1000,0)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(500,500,0, Avoidance=False)
            robot_Instance.Set_Robot_Speed(150, Transmission= True)
            robot_Instance.Reverse_Collector()
            robot_Instance.Clear_Read_Buffer()
            robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
            robot_Instance.Set_Robot_Speed(500, Transmission= True)
            robot_Instance.Start_Collector()
            robot_Instance.Clear_Read_Buffer()
            #Reverse collector
            robot_Instance.Reset_Bottle_Collected()

    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(1000,1000,0)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(500,500,0, Avoidance=False)
    robot_Instance.Set_Robot_Speed(150, Transmission= True)
    robot_Instance.Reverse_Collector()
    robot_Instance.Clear_Read_Buffer()
    robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Clear_Read_Buffer()
    #Reverse collector
    robot_Instance.Reset_Bottle_Collected()

('G1 R640 L600 T20000\r\n', 'M3 H2 S15\r\n')
PreParse  M0 X4141.96 Y298.09 A-209.29 T1 S0

Message  M0 X4141.96 Y298.09 A-209.29 T1 S0

PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
('G1 R640 L600 T20000\r\n', 'M3 H2 S15\r\n')
PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
('G1 R100 L100 T800\r\n', 'M3 H2 S0\r\n')
PreParse  M0 H2 T1

PostParse <Result ('1',) {}>
Message  M0 H2 T1

answer Timeout
Setting pos
G92 X200 Y300 A180 

Message OK: D_Profil S=150 A=50 B=100 

{'h': 1, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 1, 'b': 1, 'm': 0, 'c': 0, 'g': 1, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X920.91 Y191.87 A171.62 T2 S706

Blocked
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 1, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X1615.33 Y95.92 A156.49 T1 S512

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X1638.14 Y251.47 A43.3

KeyboardInterrupt: 

# Reset Downhill


In [11]:
robot_Instance.Go_To_Speed(120,100,10000, Detection=True)#Downhill
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7800,Y_Cal=300,A_Cal=-90, Detection=False, Calibration=True)
robot_Instance.Go_To(7800,700,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,500,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(100,100,800,X_Cal=200,Y_Cal=300,A_Cal=180, Detection=False, Calibration=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)

robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1400,300,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
robot_Instance.Go_To(1700,300,1,Path_Checking=False,Avoidance=False)
"""End of RAMP"""
robot_Instance.Start_Collector()

robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Enable_All_Sensors()
time.sleep(0.2)
robot_Instance.Clear_Read_Buffer()

robot_Instance.Go_To(1700,750,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1000,1000,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(500,500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)
robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
"""End of Grass"""
while True:
    robot_Instance.Empty_TCP()
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Set_Max_Try_Single_Goal(12)
    robot_Instance.Start_Collector()
    Spot_List =[[1000,2600,0,15],[6500,2600,0,15],[6500,5000,0,15],[4500,5000,0,15],[4500,7500,0,15],[3800,7500,0,15],[3800,5500,0,15],[7500,5500,0,15],[7500,2500,0,15],[3500,2500,0,15],[2500,500,0,15],[2500,4000,0,15],[7000,4000,0,15],[7000,3000,0,15],[3500,3000,0,15],[3500,1500,0,15],[700,3500,0,15],[5500,3500,0,15],[5500,4200,0,15],[700,4200,0,15]]
    for Spot in Spot_List:
        robot_Instance.Update_Robot_Position_From_Beacon()
        Bottle_N=0
        robot_Instance.Sensor_Enabled=Spot[3]
        print('Spot : ', Spot)
        robot_Instance.Clear_Read_Buffer()
        robot_Instance.Clear_Bottle_List()
        robot_Instance.Go_To(Spot[0],Spot[1],Spot[2])
        #Robot is now arrived to a position
        Data=robot_Instance.Get_Data_TCP()#Collect bottles from TCP Communication
        print('Data Received TCP:', Data)
        while Bottle_N<3:#Try to catch 3 Bottle
            List=robot_Instance.Update_Bottle_List(Data)#Update our internal lsit
            Bottle_List= robot_Instance.Get_Bottle_List()
            if Bottle_List!=[]:
                Bottle_X, Bottle_Y, Bottle_Score= robot_Instance.Get_Nearest_Bottle()
                if Bottle_X!=None and Bottle_Y!= None and Bottle_Score!=None:
                    Bottle_X_Extended, Bottle_Y_Extended, Valid=robot_Instance.Extend_Coordinate((Bottle_X, Bottle_Y),Distance=800)
                    print('Bottle selected:', Bottle_X,Bottle_Y,Bottle_Score, ' Extended', Bottle_X_Extended,Bottle_Y_Extended)
                    if Valid:
                        robot_Instance.Go_To(Bottle_X,Bottle_Y,Bottle_Score)
                        robot_Instance.Add_Bottle_Collected()
                        Bottle_N=Bottle_N+1
                    else:
                        robot_Instance.Clean_Bottle_Of_Position((Bottle_X, Bottle_Y))
                else:# No more bottle
                    print('No bottle')
                    break;
            else:
                print("Empty List")
                break;
        if(robot_Instance.Get_Bottle_Collected()>8):
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(1000,1000,0)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(500,500,0, Avoidance=False)
            robot_Instance.Set_Robot_Speed(150, Transmission= True)
            robot_Instance.Reverse_Collector()
            robot_Instance.Clear_Read_Buffer()
            robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
            robot_Instance.Set_Robot_Speed(500, Transmission= True)
            robot_Instance.Start_Collector()
            robot_Instance.Clear_Read_Buffer()
            #Reverse collector
            robot_Instance.Reset_Bottle_Collected()

    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(1000,1000,0)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(500,500,0, Avoidance=False)
    robot_Instance.Set_Robot_Speed(150, Transmission= True)
    robot_Instance.Reverse_Collector()
    robot_Instance.Clear_Read_Buffer()
    robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Clear_Read_Buffer()
    #Reverse collector
    robot_Instance.Reset_Bottle_Collected()

# Come Back Reset

In [ ]:

robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(100,100,800,X_Cal=200,Y_Cal=300,A_Cal=180, Detection=False, Calibration=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)

robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1400,300,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
robot_Instance.Go_To(1700,300,1,Path_Checking=False,Avoidance=False)
"""End of RAMP"""
robot_Instance.Start_Collector()

robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Enable_All_Sensors()
time.sleep(0.2)
robot_Instance.Clear_Read_Buffer()

robot_Instance.Go_To(1700,750,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(1000,1000,0,Path_Checking=False,Avoidance=True)
robot_Instance.Update_Robot_Position_From_Beacon()
robot_Instance.Go_To(500,500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)
robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
"""End of Grass"""
while True:
    robot_Instance.Empty_TCP()
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Set_Max_Try_Single_Goal(12)
    robot_Instance.Start_Collector()
    Spot_List =[[1000,2600,0,15],[6500,2600,0,15],[6500,5000,0,15],[4500,5000,0,15],[4500,7500,0,15],[3800,7500,0,15],[3800,5500,0,15],[7500,5500,0,15],[7500,2500,0,15],[3500,2500,0,15],[2500,500,0,15],[2500,4000,0,15],[7000,4000,0,15],[7000,3000,0,15],[3500,3000,0,15],[3500,1500,0,15],[700,3500,0,15],[5500,3500,0,15],[5500,4200,0,15],[700,4200,0,15]]
    for Spot in Spot_List:
        robot_Instance.Update_Robot_Position_From_Beacon()
        Bottle_N=0
        robot_Instance.Sensor_Enabled=Spot[3]
        print('Spot : ', Spot)
        robot_Instance.Clear_Read_Buffer()
        robot_Instance.Clear_Bottle_List()
        robot_Instance.Go_To(Spot[0],Spot[1],Spot[2])
        #Robot is now arrived to a position
        Data=robot_Instance.Get_Data_TCP()#Collect bottles from TCP Communication
        print('Data Received TCP:', Data)
        while Bottle_N<3:#Try to catch 3 Bottle
            List=robot_Instance.Update_Bottle_List(Data)#Update our internal lsit
            Bottle_List= robot_Instance.Get_Bottle_List()
            if Bottle_List!=[]:
                Bottle_X, Bottle_Y, Bottle_Score= robot_Instance.Get_Nearest_Bottle()
                if Bottle_X!=None and Bottle_Y!= None and Bottle_Score!=None:
                    Bottle_X_Extended, Bottle_Y_Extended, Valid=robot_Instance.Extend_Coordinate((Bottle_X, Bottle_Y),Distance=800)
                    print('Bottle selected:', Bottle_X,Bottle_Y,Bottle_Score, ' Extended', Bottle_X_Extended,Bottle_Y_Extended)
                    if Valid:
                        robot_Instance.Go_To(Bottle_X,Bottle_Y,Bottle_Score)
                        robot_Instance.Add_Bottle_Collected()
                        Bottle_N=Bottle_N+1
                    else:
                        robot_Instance.Clean_Bottle_Of_Position((Bottle_X, Bottle_Y))
                else:# No more bottle
                    print('No bottle')
                    break;
            else:
                print("Empty List")
                break;
        if(robot_Instance.Get_Bottle_Collected()>8):
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(1000,1000,0)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(500,500,0, Avoidance=False)
            robot_Instance.Set_Robot_Speed(150, Transmission= True)
            robot_Instance.Reverse_Collector()
            robot_Instance.Clear_Read_Buffer()
            robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
            robot_Instance.Set_Robot_Speed(500, Transmission= True)
            robot_Instance.Start_Collector()
            robot_Instance.Clear_Read_Buffer()
            #Reverse collector
            robot_Instance.Reset_Bottle_Collected()

    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(1000,1000,0)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(500,500,0, Avoidance=False)
    robot_Instance.Set_Robot_Speed(150, Transmission= True)
    robot_Instance.Reverse_Collector()
    robot_Instance.Clear_Read_Buffer()
    robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Clear_Read_Buffer()
    #Reverse collector
    robot_Instance.Reset_Bottle_Collected()

# Normal Loop Reset

In [ ]:
robot_Instance.Update_Robot_Position_From_Beacon()

In [ ]:
"""End of Grass"""
while True:
    robot_Instance.Empty_TCP()
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Set_Max_Try_Single_Goal(12)
    robot_Instance.Start_Collector()
    Spot_List =[[1000,2600,0,15],[6500,2600,0,15],[6500,5000,0,15],[4500,5000,0,15],[4500,7500,0,15],[3800,7500,0,15],[3800,5500,0,15],[7500,5500,0,15],[7500,2500,0,15],[3500,2500,0,15],[2500,500,0,15],[2500,4000,0,15],[7000,4000,0,15],[7000,3000,0,15],[3500,3000,0,15],[3500,1500,0,15],[700,3500,0,15],[5500,3500,0,15],[5500,4200,0,15],[700,4200,0,15]]
    for Spot in Spot_List:
        robot_Instance.Update_Robot_Position_From_Beacon()
        Bottle_N=0
        robot_Instance.Sensor_Enabled=Spot[3]
        print('Spot : ', Spot)
        robot_Instance.Clear_Read_Buffer()
        robot_Instance.Clear_Bottle_List()
        robot_Instance.Go_To(Spot[0],Spot[1],Spot[2])
        #Robot is now arrived to a position
        Data=robot_Instance.Get_Data_TCP()#Collect bottles from TCP Communication
        print('Data Received TCP:', Data)
        while Bottle_N<3:#Try to catch 3 Bottle
            List=robot_Instance.Update_Bottle_List(Data)#Update our internal lsit
            Bottle_List= robot_Instance.Get_Bottle_List()
            if Bottle_List!=[]:
                Bottle_X, Bottle_Y, Bottle_Score= robot_Instance.Get_Nearest_Bottle()
                if Bottle_X!=None and Bottle_Y!= None and Bottle_Score!=None:
                    Bottle_X_Extended, Bottle_Y_Extended, Valid=robot_Instance.Extend_Coordinate((Bottle_X, Bottle_Y),Distance=800)
                    print('Bottle selected:', Bottle_X,Bottle_Y,Bottle_Score, ' Extended', Bottle_X_Extended,Bottle_Y_Extended)
                    if Valid:
                        robot_Instance.Go_To(Bottle_X,Bottle_Y,Bottle_Score)
                        robot_Instance.Add_Bottle_Collected()
                        Bottle_N=Bottle_N+1
                    else:
                        robot_Instance.Clean_Bottle_Of_Position((Bottle_X, Bottle_Y))
                else:# No more bottle
                    print('No bottle')
                    break;
            else:
                print("Empty List")
                break;
        if(robot_Instance.Get_Bottle_Collected()>8):
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(2000,2000,1)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(1000,1000,0)
            robot_Instance.Update_Robot_Position_From_Beacon()
            robot_Instance.Go_To(500,500,0, Avoidance=False)
            robot_Instance.Set_Robot_Speed(150, Transmission= True)
            robot_Instance.Reverse_Collector()
            robot_Instance.Clear_Read_Buffer()
            robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
            robot_Instance.Set_Robot_Speed(500, Transmission= True)
            robot_Instance.Start_Collector()
            robot_Instance.Clear_Read_Buffer()
            #Reverse collector
            robot_Instance.Reset_Bottle_Collected()

    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(2000,2000,1)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(1000,1000,0)
    robot_Instance.Update_Robot_Position_From_Beacon()
    robot_Instance.Go_To(500,500,0, Avoidance=False)
    robot_Instance.Set_Robot_Speed(150, Transmission= True)
    robot_Instance.Reverse_Collector()
    robot_Instance.Clear_Read_Buffer()
    robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
    robot_Instance.Set_Robot_Speed(500, Transmission= True)
    robot_Instance.Clear_Read_Buffer()
    #Reverse collector
    robot_Instance.Reset_Bottle_Collected()

total time capture image:  0.6448309421539307
g  not found.
[-169.35712925529643, 63.43494882292201, -0.30970389995956754] [(0, 8000), (8000, 0), (0, 0)]
Beacon worked!
total time post math:  1.5425777435302734
G92 X375 Y4573 A266 

375.2948680963409 4572.847951080832 265.6064796101654
Spot :  [1000, 2600, 0, 15]
{'b': 0, 'i': 1, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 1, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X373.46 Y3939.62 A271.35 T2 S2304

Blocked
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X378.40 Y4072.61 A272.44 T1 S0

Timeout
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X481.61 Y3957.78 A342.91 T1 S0

Timeout
{'b': 0, 'i': 0, 'h': 0, 'f': 0, 'e': 0, 'a': 0, 'c': 0, 'm': 0, 'l': 0, 'k': 0, 'j': 0, 'd': 0, 'g': 0}
Message  M0 X980.67 Y2768.80 A276.78 T1 S0

Timeout
Data Received TCP: None
Empty List
total time capture 